In [51]:
import sys
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if path not in sys.path:
    sys.path.append(path)
import pandas as pd
import numpy as np
from custom_transformers.CustomTransformers import Cleaner , ReplyFinder , QuestionClassifier ,InformationClassifier
from custom_transformers.CustomTransformers import CombineMessageReply  
from custom_transformers.CustomTransformers import GetLocationList,GetStatusList,FilterIsGivingInformation
from custom_transformers.CustomTransformers import FilterHasLoc,FilterHasStat,GenerateSummary,NerClassifier
from constants import COLUMNS_WE_NEED
from utils.transform_data import transform_reply_to
from utils.clean_data import clean_text
from sklearn.pipeline import Pipeline



In [52]:
df = pd.read_csv('../data/raw.csv')

In [53]:
df.head(1)

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period
0,Message,428911,"{'_': 'PeerChannel', 'channel_id': 1756020315}",2023-03-17 11:04:46+00:00,الدي سي اوه عليه مشاكل ؟؟,False,False,False,False,False,...,[],NaN,NaN,"{'_': 'MessageReplies', 'replies': 0, 'replies...",NaN,NaN,NaN,NaN,[],86400.0


In [54]:
df = df.sample(5)

In [55]:
df['message']  = "حاجز  المنوفية مسكر"

In [56]:
from sklearn.base import BaseEstimator, TransformerMixin
import sys
import os
import joblib

from utils.helpers import combine_sub_words, word2features
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if path not in sys.path:
    sys.path.append(path)
from keras.models import load_model
   
from utils.clean_data import clean_text
from constants import COLUMNS_WE_NEED
from utils.transform_data import transform_reply_to
import tensorflow as tf
import pyarabic.araby as araby
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline



In [57]:
pep = Pipeline([('cleaner', Cleaner()), ('reply_finder', ReplyFinder()),
                     ("question_classifier",QuestionClassifier()),("information_classifier",InformationClassifier())
                     ,("combine_message_reply",CombineMessageReply()),("filter",FilterIsGivingInformation()),
                     ("ner_crf_classifier",NerClassifier()),("Locations",GetLocationList()),("Status",GetStatusList()),
                     ("filter_has_loc",FilterHasLoc()),("filter_has_stat",FilterHasStat()),("summary",GenerateSummary())])


In [58]:
result = pep.fit_transform(df)

Current step: Cleaner
Current step: ReplyFinder
Current step: QuestionClassifier
1/1 [==============================] - 0s 16ms/step
Current step: InformationClassifier
1/1 [==============================] - 0s 438ms/step
Current step: CombineMessageReply
Current step: FilterIsGivingInformation
NerClassifier
hi
1/1 [==============================] - 1s 1s/step
Current step: GetLocationList
Current step: GetStatusList
Current step: FilterHasLoc
Current step: FilterHasStat
Current step: GenerateSummary
(5, 4)
(5, 4)
(5, 4)


In [59]:
result

array([{'location': 'المنوفية ', 'status': 'مسكر '},
       {'location': 'المنوفية ', 'status': 'مسكر '},
       {'location': 'المنوفية ', 'status': 'مسكر '},
       {'location': 'المنوفية ', 'status': 'مسكر '},
       {'location': 'المنوفية ', 'status': 'مسكر '}], dtype=object)

In [60]:
# convert result to dataframe with two columns: location and status
dd = pd.DataFrame(result)

In [61]:
dd.sample(10)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
# dd.to_csv('../data/data_for_ner.csv',index=False)

In [ ]:
# # dd = dd[['full_text','locs','Stat','summary']]
# # # # get the rows contains more than one location
# # # dd = dd[dd['locs'].apply(lambda x: len(x)) > 1]
# # pd.set_option('display.max_colwidth', None)
# # dd.sample(5)
# dd


In [ ]:
# visulize the pipeline
from sklearn import set_config
set_config(display='diagram')
pep



Pipeline(steps=[('cleaner', Cleaner()), ('reply_finder', ReplyFinder()),
                ('question_classifier', QuestionClassifier()),
                ('information_classifier', InformationClassifier()),
                ('combine_message_reply', CombineMessageReply()),
                ('filter', FilterIsGivingInformation()),
                ('ner_crf_classifier', NerClassifier()),
                ('Locations', GetLocationList()), ('Status', GetStatusList()),
                ('filter_has_loc', FilterHasLoc()),
                ('filter_has_stat', FilterHasStat()),
                ('summary', GenerateSummary())])

In [ ]:
# new_df = pd.read_csv('../data/new2.csv')
# new = pipeline.fit_transform(new_df)


In [ ]:
# pd.set_option('display.max_colwidth', None)
# # filter out the rows that LOC is empty and STAT is empty
# new = new[(new['LOC'] != "") & (new['STAT'] != "")]
# new[['full_text','LOC','STAT']].sample(10)

In [ ]:
# row = new.iloc[0]
# print(row['full_text'])
# print(row['ner'])

In [ ]:
# get_LOC(row)

In [ ]:
# # dd = pipeline.fit_transform(df)

In [ ]:
# pd.set_option('display.max_colwidth', None)
# dd.sample(10)
# # see the distribution of the hajez names


In [ ]:
# # in dd get any row that the len of full_text.split() != len of ner.split()
# dd[dd['full_text'].apply(lambda x: len(x.split())) == dd['ner'].apply(lambda x: len(x.split()))]

In [ ]:
# # print sample from the dataframe each word with the predicted named entity
# random_sample = dd.sample(1)
# # print thim in this way  word: named entity
# for word, ner in zip(random_sample['full_text'].values[0].split(), random_sample['ner'].values[0].split()):
#     print(f"{word}: {ner}")


In [ ]:
# dd.to_csv('../data/data_for_hajez_classifier.csv',index=False)